# Purity Reviewer Example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

sys.path.append(os.path.abspath( os.path.join("..") ))

from PurityReviewer.Reviewers.MatchedPurityReviewer import MatchedPurityReviewer
from PurityReviewer.Reviewers.ManualPurityReviewer import ManualPurityReviewer
from PurityReviewer.Reviewers.PrecalledPurityReviewer import PrecalledPurityReviewer
from PurityReviewer.AppComponents.utils import download_rdata
import pandas as pd
import numpy as np
import os

# Run with simuated tumor data
## Setup

Clone the `SimulatedTumorData` repo and install in your environment.
```
cd <path to put SimulatedTumorData repo>
git clone https://github.com/getzlab/SimulatedTumorData

conda activate <your env>
pip install -e <path to put SimulatedTumorData repo>
```

This only needs to be done once.

## Load simulated data

In [3]:
# from SimulatedTumorData.src.generate_simulated_data import load_patients_and_samples

In [4]:
# samples, participants = load_patients_and_samples(path_to_sim_data="SimulatedTumorData/sim_data")

## Pass in data

In [5]:
# from PurityReviewer.AppComponents.utils import parse_absolute_soln, CSIZE_DEFAULT
# from PurityReviewer.AppComponents.utils import parse_absolute_soln_simulatedTumorData

In [6]:
# data_path = 'simulated_data_purity_review'

In [7]:
# matched_reviewer = MatchedPurityReviewer()
# matched_reviewer.set_review_data(
#     data_path=data_path, 
#     description='Matched purity reviewer for simulated data', 
#     df=samples,
#     index=samples.index, 
# )
# matched_reviewer.set_review_app(
#     sample_info_cols=['ABSOLUTE_plot_fn', 'wxs_purity', 'wxs_ploidy'],
#     acs_col='cnv_seg_fn', 
#     maf_col='maf_fn',
#     rdata_fn_col='ABSOLUTE_RData',
#     mut_fig_hover_data=['Hugo_Symbol', 'Chromosome', 'Start_position'],
#     csize=CSIZE_DEFAULT,
#     custom_parse_absolute_soln=parse_absolute_soln_simulatedTumorData # <-- update with my_custom_parse_absolute_soln()
# )

# matched_reviewer.set_default_review_data_annotations_configuration()
# matched_reviewer.set_default_autofill()

## Run the reviewer

In [8]:
# matched_reviewer.run(port=8099, mode='tab', collapsable=False, hide_history_df_cols=['source_data_fn'])

In [9]:
# matched_reviewer.get_annot()

# Purity Review from CGA characterization pipeline Terra workspace

## Load data

In [10]:
import dalmatian

```
import os
os.environ["GCLOUD_PROJECT"] = <google project>

wm = dalmatian.WorkspaceManager(<namespace>/<workspace_name>)
pairs_df = wm.get_pairs()

# Download rdata from ABSOLUTE locally
rdata_dir = <directory to save rData>
downloaded_rdata_s = download_rdata(pairs_df['absolute_rdata'], rdata_dir=rdata_dir)
downloaded_rdata_s.name = 'local_absolute_rdata'
pairs_df = pd.concat([pairs_df, downloaded_rdata_s], axis=1)
```

In [11]:
workspace = 'broad-tcga-wgs-terra/TCGA_BRCA_WGS'
wm = dalmatian.WorkspaceManager(workspace)
wm_pairs_df = wm.get_pairs()
wm_pairs_df = wm_pairs_df[wm_pairs_df['absolute_rdata_WGS'].notna()] #.set_index('tumor_submitter_id')
display(wm_pairs_df)

,participant,tumor_type_code,normal_type_code,case_sample,control_sample,normal_submitter_id,tumor_submitter_id,tumor_type,normal_type,T_picard_alignment_summary_hist_WGS,...,T_mean_read_length_WGS,T_het_sensitivity_WGS,T_mismatch_rate_WGS,CrossCheck_min_LOD_value,N_mean_coverage_WGS,T_median_insert_size_WGS,N_pct_chimeras_WGS,CrossCheck_min_LOD_lanes,T_mean_coverage_WGS,N_mean_read_length_WGS
pair_id,,,,,,,,,,,,,,,,,,,,,
0f6ed829-a59b-48b7-a2ca-3aa9db40e4c2_4c882f6b-fd94-43b7-a6e0-2ef467da9884,27c1c094-690b-4973-8900-a797ad88f98c,TP,NB,0f6ed829-a59b-48b7-a2ca-3aa9db40e4c2,4c882f6b-fd94-43b7-a6e0-2ef467da9884,TCGA-D8-A27T-10A,TCGA-D8-A27T-01A,Primary Tumor,Blood Derived Normal,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
317b72a8-0332-4db8-b9d1-9a26d9907a24_ffa2ca7e-93d7-452d-870e-c7c36c032e4f,ff0f875f-113a-4c0f-8c0a-4259cd496e6d,TP,NB,317b72a8-0332-4db8-b9d1-9a26d9907a24,ffa2ca7e-93d7-452d-870e-c7c36c032e4f,TCGA-C8-A26W-10A,TCGA-C8-A26W-01A,Primary Tumor,Blood Derived Normal,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,...,151,0.89869,0.004736,70.145458,39.437779,406,0.034528,TCGA-C8-A26W-01A-11D-A89A-36.WholeGenome.RP-16...,71.227847,151
4584c7e2-8011-4231-a5e6-26a264f6d1b1_015965ba-53b4-4b89-bae3-5dc351212d9f,9ced0c70-c46c-4f98-8f48-840defe4b8b1,TP,NB,4584c7e2-8011-4231-a5e6-26a264f6d1b1,015965ba-53b4-4b89-bae3-5dc351212d9f,TCGA-D8-A1JA-10A,TCGA-D8-A1JA-01A,Primary Tumor,Blood Derived Normal,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,...,151,0.904808,0.004527,82.564876,35.604842,387,0.042127,TCGA-D8-A1JA-01A-11D-A897-36.WholeGenome.RP-16...,88.668252,151
45c79141-a125-4305-b1b7-c838de1af8a3_574293a5-ec7b-479d-af12-27b5e1aaeca1,214a4507-d974-4b3e-8525-7408fccc6a0f,TP,NB,45c79141-a125-4305-b1b7-c838de1af8a3,574293a5-ec7b-479d-af12-27b5e1aaeca1,TCGA-BH-A0B7-10A,TCGA-BH-A0B7-01A,Primary Tumor,Blood Derived Normal,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4874e4e2-c067-490d-9d99-8d3ca194b0a4_de02562d-cbcf-434e-81a3-2eb7f09a3747,dd0f8e5a-383d-4f03-bb71-0e115d4e442c,TP,NB,4874e4e2-c067-490d-9d99-8d3ca194b0a4,de02562d-cbcf-434e-81a3-2eb7f09a3747,TCGA-C8-A12M-10A,TCGA-C8-A12M-01A,Primary Tumor,Blood Derived Normal,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4c54a99b-a820-4602-9b39-deaca98034a5_0ab0bff3-9c3d-466b-91c2-5c95613a5073,68177e92-4784-45d9-af14-4ce0c00eae74,TP,NB,4c54a99b-a820-4602-9b39-deaca98034a5,0ab0bff3-9c3d-466b-91c2-5c95613a5073,TCGA-A8-A08J-10B,TCGA-A8-A08J-01A,Primary Tumor,Blood Derived Normal,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4f69c5f3-4e8c-4b71-a5fe-3179412c17e6_a91eabd7-bfbc-43f4-ae3a-46800dedce29,d5be2ab3-f748-480d-954b-385cf01b4c81,TP,NB,4f69c5f3-4e8c-4b71-a5fe-3179412c17e6,a91eabd7-bfbc-43f4-ae3a-46800dedce29,TCGA-D8-A27N-10A,TCGA-D8-A27N-01A,Primary Tumor,Blood Derived Normal,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,...,151,0.874941,0.005474,90.823794,34.502734,406,0.037964,TCGA-D8-A27N-01A-11D-A89A-36.WholeGenome.RP-16...,74.300717,151
58bfe278-a80f-4722-b286-4d966214d244_b41594a0-5892-4ae6-a354-ddf09a7772a0,e4fc0909-f284-4471-866d-d8967b6adcbc,TP,NB,58bfe278-a80f-4722-b286-4d966214d244,b41594a0-5892-4ae6-a354-ddf09a7772a0,TCGA-E2-A14P-10A,TCGA-E2-A14P-01A,Primary Tumor,Blood Derived Normal,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5de50617-0a03-45b7-8ffd-bcf3a0680d6b_aaca1df7-d409-4e6e-b592-f035409fcb27,89e867de-d30e-4bfc-a453-0a76309dabd5,TP,NB,5de50617-0a03-45b7-8ffd-bcf3a0680d6b,aaca1df7-d409-4e6e-b592-f035409fcb27,TCGA-C8-A132-10A,TCGA-C8-A132-01A,Primary Tumor,Blood Derived Normal,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# wm_pairs_df['absolute_rdata_WGS'].head()[0]

In [13]:
# # check the version of google cloud and dalmatian to see if they're compatible
# file_to_download_path = 'gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880de1cd/pair/0f6ed829-a59b-48b7-a2ca-3aa9db40e4c2_4c882f6b-fd94-43b7-a6e0-2ef467da9884/ABSOLUTE__2024-07-25--21-05-28_ltrxn0a_rpp01pa_zlsikrrppbm1g/jobs/0/workspace/TCGA-D8-A27T-01A.PP-modes.data.RData'
# rdata_dir = 'local_rdata'
# pair_id = 'Test_pair_data1'
# full_local_path = f'{rdata_dir}/{pair_id}.absolute.rdata'

# dalmatian.getblob(file_to_download_path).download_to_filename(full_local_path)

In [14]:
# # download rdata locally
## error with trying to access the r data from the wm_pairs_df dataframe because 'local_absolute_rdata' column is never defined
# local_rdata_dir = 'local_rdata'
# wm_pairs_df['local_absolute_rdata'] = download_rdata(wm_pairs_df['absolute_rdata_WGS'], rdata_dir=local_rdata_dir)

In [15]:
# wm_pairs_df['local_absolute_rdata'].iloc[0]

In [16]:
wm_samples_df = wm.get_samples()
wm_samples_df

,PCA__ABSOLUTE__Cancer_DNA_fraction,PCA__ABSOLUTE__Coverage_for_80_pct_power,PCA__ABSOLUTE__Genome_doublings,PCA__ABSOLUTE__Subclonal_genome_fraction,PCA__ABSOLUTE__call_status,PCA__ABSOLUTE__ploidy,PCA__ABSOLUTE__purity,PCA__ABSOLUTE__sample,PCA__ABSOLUTE__solution,WGS__BWAwithMarkDuplicatesandBQSR__aligned_reads__bai__drs_url,WGS__BWAwithMarkDuplicatesandBQSR__aligned_reads__bam__average_read_length,WGS__BWAwithMarkDuplicatesandBQSR__aligned_reads__bam__drs_url,WGS__BWAwithMarkDuplicatesandBQSR__aligned_reads__bam__gdc_release,WGS__BWAwithMarkDuplicatesandBQSR__aligned_reads__bam__mean_coverage,participant,sample_type,sample_type_code,submitter_id,tissue_type
sample_id,,,,,,,,,,,,,,,,,,,
000002fc-53a0-420e-b2aa-a40a358bba37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drs://dg.4DFC:d7fcc34b-163a-4df0-9a8c-3cc4e97f...,151,drs://dg.4DFC:72922f34-f7bd-4d66-b099-b25cfbb3...,39,37.374038,271e8024-a008-4bf0-9f6e-177f60096b1b,Blood Derived Normal,NB,TCGA-E2-A15G-10A,Normal
0039a298-3369-4da5-9bfd-19792e9632be,0.73,11.0,0.0,0.14,called,2.01,0.73,TCGA-OL-A5RY-01A-21D-A28A-01,new,drs://dg.4DFC:c56ff381-9b9a-4d82-8547-1acf6d68...,151,drs://dg.4DFC:81f41338-0539-45d4-a656-245ee983...,39,87.247114,8360cec6-daf6-41c9-9a4f-7fc03c958dcc,Primary Tumor,TP,TCGA-OL-A5RY-01A,Tumor
003c8cff-ec0e-4f7c-b38b-93575fd68a23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drs://dg.4DFC:ec3ff1b7-ecc9-439e-b716-18b9294e...,151,drs://dg.4DFC:ca29a575-d626-49e6-8519-c74947ba...,39,32.350460,4e6edfe6-adcb-4c12-8ff4-38a79f5887e8,Blood Derived Normal,NB,TCGA-B6-A0X5-10A,Normal
0050d7c9-ece9-4b6c-8023-1ff2efcb3c9c,0.73,17.0,1.0,0.00,called,3.05,0.64,TCGA-3C-AALI-01A-11D-A41E-01,new,drs://dg.4DFC:39da1845-c58a-4b0c-abc0-3dfa06d4...,151,drs://dg.4DFC:3b30c210-c8e8-49e2-a756-3fac591d...,39,95.615588,55262fcb-1b01-4480-b322-36570430c917,Primary Tumor,TP,TCGA-3C-AALI-01A,Tumor
008dafdd-a3d1-4801-8c0a-8714e2b58e48,0.50,16.0,0.0,0.02,called,1.95,0.51,TCGA-AO-A0JJ-01A-11D-A059-01,new,drs://dg.4DFC:22929a72-0afa-4967-87fc-8461e09f...,151,drs://dg.4DFC:c7125f93-1afd-449e-bbfe-c9c2abd9...,39,93.925069,bef7b135-a727-45e8-850a-cc4cd56c49aa,Primary Tumor,TP,TCGA-AO-A0JJ-01A,Tumor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fed3a891-21be-4196-b9ef-0bf7c8f976aa,0.64,11.0,0.0,0.45,called,1.80,0.67,TCGA-GM-A3XL-01A-11D-A22W-01,new,drs://dg.4DFC:469e6cf6-5f73-4469-bcdf-1d16307b...,151,drs://dg.4DFC:3c3e329e-b43d-4068-b70d-13cef90d...,39,92.007449,65cac997-4d39-4501-85ec-4fcb328a8eb5,Primary Tumor,TP,TCGA-GM-A3XL-01A,Tumor
fef9fd2b-5bab-43eb-aa47-1e93adf29099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drs://dg.4DFC:b653d0fd-8363-4ac8-9305-4df35f09...,151,drs://dg.4DFC:d9bd1bf3-37c4-4ee8-a4f1-0e74220f...,39,36.809534,807b62ed-d3f7-4211-83be-754386ff2c96,Blood Derived Normal,NB,TCGA-E9-A3HO-10A,Normal
ff8c73fc-188d-4e36-867c-afdb6fa7b1d8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drs://dg.4DFC:2e632563-7540-42ac-a5bd-8f2c64fa...,151,drs://dg.4DFC:7c71a45b-ab32-4944-b3ce-bdb36c17...,39,40.638454,a6e11b30-3ae8-4dd1-b04c-a730c6a79746,Blood Derived Normal,NB,TCGA-AR-A1AH-10A,Normal


In [17]:
precalled_purities_df = wm_samples_df[['PCA__ABSOLUTE__Cancer_DNA_fraction']]
precalled_purities_df = precalled_purities_df[precalled_purities_df['PCA__ABSOLUTE__Cancer_DNA_fraction'].notna()]
precalled_purities_df

,PCA__ABSOLUTE__Cancer_DNA_fraction
sample_id,
0039a298-3369-4da5-9bfd-19792e9632be,0.73
0050d7c9-ece9-4b6c-8023-1ff2efcb3c9c,0.73
008dafdd-a3d1-4801-8c0a-8714e2b58e48,0.50
00a59c97-91b4-413e-8d9e-674753997221,0.54
00f81e98-e5c3-43b2-9757-f769ec17f3f8,0.61
...,...
fd735dc5-2ec3-43bd-b1b1-8fa811be0633,0.90
fe0ddcfc-f6e9-460d-b8e6-5187ffe8eb4f,0.50
fe3921db-1691-432f-b7fa-703ae8d7bfd2,0.51


## Pass in data

In [18]:
from PurityReviewer.AppComponents.utils import CSIZE_DEFAULT
sex_chr_map = {'23': 'X', '24': 'Y'}
rename_chroms = {x: sex_chr_map[x] if x in sex_chr_map.keys() else x for x in CSIZE_DEFAULT.keys()}
custom_csize = {f'chr{rename_chroms[chrom]}': length for chrom, length in CSIZE_DEFAULT.items()}
custom_csize

{'chr1': 249250621,
 'chr2': 243199373,
 'chr3': 198022430,
 'chr4': 191154276,
 'chr5': 180915260,
 'chr6': 171115067,
 'chr7': 159138663,
 'chr8': 146364022,
 'chr9': 141213431,
 'chr10': 135534747,
 'chr11': 135006516,
 'chr12': 133851895,
 'chr13': 115169878,
 'chr14': 107349540,
 'chr15': 102531392,
 'chr16': 90354753,
 'chr17': 81195210,
 'chr18': 78077248,
 'chr19': 59128983,
 'chr20': 63025520,
 'chr21': 48129895,
 'chr22': 51304566,
 'chrX': 156040895,
 'chrY': 57227415}

In [19]:
from rpy2.robjects import r, pandas2ri
import rpy2.robjects as robjects

def my_custom_parse_absolute_soln(rdata_path: str): # has to be a local path   
    absolute_rdata_cols = ['alpha', 'tau', 'tau_hat', '0_line', '1_line',
                       'sigma_H', 
                       'theta_Q', 
                       'lambda',  
                       'SCNA_likelihood', 
                       'Kar_likelihood', 
                       'SSNVs_likelihood']
    pandas2ri.activate()
    print('New version!')
    r_list_vector = robjects.r['load'](rdata_path)
    r_list_vector = robjects.r[r_list_vector[0]]
    # r_data_id = r_list_vector.names[0]
    # rdata_tables = r_list_vector.rx2(str(r_data_id))
    
    # mode_res = rdata_tables.rx2('mode.res')
    # mode_tab = mode_res.rx2('mode.tab')
    mode_res = r_list_vector.rx2('mode.res')
    mode_tab = r_list_vector.rx2('mode.tab') # or segtab?
    mod_tab_df = pd.DataFrame(columns=absolute_rdata_cols)
    mod_tab_df['alpha'] = mode_tab[:, 0]
    mod_tab_df['tau'] = mode_tab[:, 1]
    mod_tab_df['tau_hat'] = mode_tab[:, 7]
    mod_tab_df['0_line'] = mode_tab[:, 3]
    mod_tab_df['step_size'] = mode_tab[:, 4] * 2
    mod_tab_df['1_line'] = mod_tab_df['step_size'] + mod_tab_df['0_line']
    mod_tab_df['sigma_H'] = mode_tab[:, 8]
    mod_tab_df['theta_Q'] = mode_tab[:, 11]
    mod_tab_df['lambda'] = mode_tab[:, 12]
    mod_tab_df['SCNA_likelihood'] = mode_tab[:, 15]
    mod_tab_df['Kar_likelihood'] = mode_tab[:, 17]
    mod_tab_df['SSNVs_likelihood'] = mode_tab[:, 20]

    return mod_tab_df

In [20]:
wm_pairs_df["case_sample"]

pair_id
0f6ed829-a59b-48b7-a2ca-3aa9db40e4c2_4c882f6b-fd94-43b7-a6e0-2ef467da9884    0f6ed829-a59b-48b7-a2ca-3aa9db40e4c2
317b72a8-0332-4db8-b9d1-9a26d9907a24_ffa2ca7e-93d7-452d-870e-c7c36c032e4f    317b72a8-0332-4db8-b9d1-9a26d9907a24
4584c7e2-8011-4231-a5e6-26a264f6d1b1_015965ba-53b4-4b89-bae3-5dc351212d9f    4584c7e2-8011-4231-a5e6-26a264f6d1b1
45c79141-a125-4305-b1b7-c838de1af8a3_574293a5-ec7b-479d-af12-27b5e1aaeca1    45c79141-a125-4305-b1b7-c838de1af8a3
4874e4e2-c067-490d-9d99-8d3ca194b0a4_de02562d-cbcf-434e-81a3-2eb7f09a3747    4874e4e2-c067-490d-9d99-8d3ca194b0a4
4c54a99b-a820-4602-9b39-deaca98034a5_0ab0bff3-9c3d-466b-91c2-5c95613a5073    4c54a99b-a820-4602-9b39-deaca98034a5
4f69c5f3-4e8c-4b71-a5fe-3179412c17e6_a91eabd7-bfbc-43f4-ae3a-46800dedce29    4f69c5f3-4e8c-4b71-a5fe-3179412c17e6
58bfe278-a80f-4722-b286-4d966214d244_b41594a0-5892-4ae6-a354-ddf09a7772a0    58bfe278-a80f-4722-b286-4d966214d244
5de50617-0a03-45b7-8ffd-bcf3a0680d6b_aaca1df7-d409-4e6e-b592-f035409fcb27    5de

In [21]:
# # case_sample -> tumor sample -> in wm_pairs_df
# # sample_id -> sample_df
# # precalled_purities_df -> precalled_purity df that has sample_id and precalled purity value
#     # sample_id, PCA__ABSOLUTE__Cancer_DNA_fraction
# # wm_pairs_df
#     # "case_sample"

# import pandas as pd

# def merge_pairs_precalled_purities_df(pairs_df, precalled_purity_df):
#     """
#     Merges the pairs dataframe with the precalled purity dataframe based on the tumor sample id

#     pairs_df: df, dataframe of the pairs sample data

#     precalled_purity_df: df, dataframe of the precalled purity values with sample id

#     Returns:
#         Merged dataframe with the precalled purity values added as a new column to the original pairs_df dataframe
#     """

#     pairs_df = pairs_df.copy()
#     precalled_purity_df = precalled_purity_df.copy()
        

#     return pairs_df.merge(precalled_purity_df, how="inner", left_on="case_sample", right_on="sample_id").set_index('tumor_submitter_id')

In [22]:
from PurityReviewer.AppComponents.utils import merge_pairs_precalled_purities_df

print("length before merging: ", len(wm_pairs_df))
wm_pairs_df = merge_pairs_precalled_purities_df(wm_pairs_df, precalled_purities_df)
print("length after merging: ", len(wm_pairs_df))

print("Merged successfully: ", "case_sample" in wm_pairs_df.columns and "PCA__ABSOLUTE__Cancer_DNA_fraction" in wm_pairs_df.columns)
print(wm_pairs_df[["case_sample", "PCA__ABSOLUTE__Cancer_DNA_fraction"]])
print()
print("Is tumor submitter id in wm_pairs_df", 'tumor_submitter_id' in wm_pairs_df.columns)

length before merging:  21
length after merging:  21
Merged successfully:  True
                                             case_sample  \
tumor_submitter_id                                         
TCGA-D8-A27T-01A    0f6ed829-a59b-48b7-a2ca-3aa9db40e4c2   
TCGA-C8-A26W-01A    317b72a8-0332-4db8-b9d1-9a26d9907a24   
TCGA-D8-A1JA-01A    4584c7e2-8011-4231-a5e6-26a264f6d1b1   
TCGA-BH-A0B7-01A    45c79141-a125-4305-b1b7-c838de1af8a3   
TCGA-C8-A12M-01A    4874e4e2-c067-490d-9d99-8d3ca194b0a4   
TCGA-A8-A08J-01A    4c54a99b-a820-4602-9b39-deaca98034a5   
TCGA-D8-A27N-01A    4f69c5f3-4e8c-4b71-a5fe-3179412c17e6   
TCGA-E2-A14P-01A    58bfe278-a80f-4722-b286-4d966214d244   
TCGA-C8-A132-01A    5de50617-0a03-45b7-8ffd-bcf3a0680d6b   
TCGA-D8-A140-01A    660954c6-51ed-4b74-8eb0-65c282da3971   
TCGA-A8-A07B-01A    685fa2ea-3dff-450a-8ea6-e97dfe9000a4   
TCGA-A2-A0T3-01A    78f8b01e-292f-4131-a3ef-4f9faec7913a   
TCGA-A8-A09I-01A    a09a99f9-33e3-45d5-bfc2-f57ce149a527   
TCGA-EW-A1OZ-01A    

In [23]:
wm_pairs_df

,participant,tumor_type_code,normal_type_code,case_sample,control_sample,normal_submitter_id,tumor_type,normal_type,T_picard_alignment_summary_hist_WGS,N_picard_gc_bias_metrics_WGS,...,T_het_sensitivity_WGS,T_mismatch_rate_WGS,CrossCheck_min_LOD_value,N_mean_coverage_WGS,T_median_insert_size_WGS,N_pct_chimeras_WGS,CrossCheck_min_LOD_lanes,T_mean_coverage_WGS,N_mean_read_length_WGS,PCA__ABSOLUTE__Cancer_DNA_fraction
tumor_submitter_id,,,,,,,,,,,,,,,,,,,,,
TCGA-D8-A27T-01A,27c1c094-690b-4973-8900-a797ad88f98c,TP,NB,0f6ed829-a59b-48b7-a2ca-3aa9db40e4c2,4c882f6b-fd94-43b7-a6e0-2ef467da9884,TCGA-D8-A27T-10A,Primary Tumor,Blood Derived Normal,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.63
TCGA-C8-A26W-01A,ff0f875f-113a-4c0f-8c0a-4259cd496e6d,TP,NB,317b72a8-0332-4db8-b9d1-9a26d9907a24,ffa2ca7e-93d7-452d-870e-c7c36c032e4f,TCGA-C8-A26W-10A,Primary Tumor,Blood Derived Normal,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,...,0.89869,0.004736,70.145458,39.437779,406,0.034528,TCGA-C8-A26W-01A-11D-A89A-36.WholeGenome.RP-16...,71.227847,151,0.63
TCGA-D8-A1JA-01A,9ced0c70-c46c-4f98-8f48-840defe4b8b1,TP,NB,4584c7e2-8011-4231-a5e6-26a264f6d1b1,015965ba-53b4-4b89-bae3-5dc351212d9f,TCGA-D8-A1JA-10A,Primary Tumor,Blood Derived Normal,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,...,0.904808,0.004527,82.564876,35.604842,387,0.042127,TCGA-D8-A1JA-01A-11D-A897-36.WholeGenome.RP-16...,88.668252,151,0.77
TCGA-BH-A0B7-01A,214a4507-d974-4b3e-8525-7408fccc6a0f,TP,NB,45c79141-a125-4305-b1b7-c838de1af8a3,574293a5-ec7b-479d-af12-27b5e1aaeca1,TCGA-BH-A0B7-10A,Primary Tumor,Blood Derived Normal,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.25
TCGA-C8-A12M-01A,dd0f8e5a-383d-4f03-bb71-0e115d4e442c,TP,NB,4874e4e2-c067-490d-9d99-8d3ca194b0a4,de02562d-cbcf-434e-81a3-2eb7f09a3747,TCGA-C8-A12M-10A,Primary Tumor,Blood Derived Normal,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.82
TCGA-A8-A08J-01A,68177e92-4784-45d9-af14-4ce0c00eae74,TP,NB,4c54a99b-a820-4602-9b39-deaca98034a5,0ab0bff3-9c3d-466b-91c2-5c95613a5073,TCGA-A8-A08J-10B,Primary Tumor,Blood Derived Normal,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.73
TCGA-D8-A27N-01A,d5be2ab3-f748-480d-954b-385cf01b4c81,TP,NB,4f69c5f3-4e8c-4b71-a5fe-3179412c17e6,a91eabd7-bfbc-43f4-ae3a-46800dedce29,TCGA-D8-A27N-10A,Primary Tumor,Blood Derived Normal,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,...,0.874941,0.005474,90.823794,34.502734,406,0.037964,TCGA-D8-A27N-01A-11D-A89A-36.WholeGenome.RP-16...,74.300717,151,0.59
TCGA-E2-A14P-01A,e4fc0909-f284-4471-866d-d8967b6adcbc,TP,NB,58bfe278-a80f-4722-b286-4d966214d244,b41594a0-5892-4ae6-a354-ddf09a7772a0,TCGA-E2-A14P-10A,Primary Tumor,Blood Derived Normal,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.55
TCGA-C8-A132-01A,89e867de-d30e-4bfc-a453-0a76309dabd5,TP,NB,5de50617-0a03-45b7-8ffd-bcf3a0680d6b,aaca1df7-d409-4e6e-b592-f035409fcb27,TCGA-C8-A132-10A,Primary Tumor,Blood Derived Normal,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,gs://fc-secure-df24feb9-b120-454d-98f1-2ac4880...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.40


In [24]:
local_rdata_dict = dict()
rdata_dir = "local_rdata"
pair_ids = ["TCGA-A2-A0T3-01A", "TCGA-A8-A0A7-01A", "TCGA-A8-A07B-01A", "TCGA-A8-A08J-01A",
            "TCGA-A8-A09I-01A", "TCGA-AN-A049-01A", "TCGA-AO-A12D-01A", "TCGA-AR-A0TQ-01A",
            "TCGA-BH-A0B7-01A", "TCGA-C8-A12L-01A", "TCGA-C8-A12M-01A", "TCGA-C8-A26W-01A", 
            "TCGA-C8-A132-01A", "TCGA-D8-A1JA-01A", "TCGA-D8-A27N-01A", "TCGA-D8-A27T-01A",
            "TCGA-D8-A140-01A", "TCGA-E2-A14P-01A", "TCGA-EW-A1IW-01A", "TCGA-EW-A1OZ-01A",
            "TCGA-EW-A1PH-01A"]

for pair_id in pair_ids:
    local_absolute_rdata_fn = f'{rdata_dir}/{pair_id}.RData'
    local_rdata_dict[pair_id] = local_absolute_rdata_fn
# wm_pairs_df.set_index('tumor_submitter_id', inplace=True)
# print(pd.Series(local_rdata_dict))
wm_pairs_df['local_absolute_rdata'] = pd.Series(local_rdata_dict)
print(wm_pairs_df['local_absolute_rdata'])

tumor_submitter_id
TCGA-D8-A27T-01A    local_rdata/TCGA-D8-A27T-01A.RData
TCGA-C8-A26W-01A    local_rdata/TCGA-C8-A26W-01A.RData
TCGA-D8-A1JA-01A    local_rdata/TCGA-D8-A1JA-01A.RData
TCGA-BH-A0B7-01A    local_rdata/TCGA-BH-A0B7-01A.RData
TCGA-C8-A12M-01A    local_rdata/TCGA-C8-A12M-01A.RData
TCGA-A8-A08J-01A    local_rdata/TCGA-A8-A08J-01A.RData
TCGA-D8-A27N-01A    local_rdata/TCGA-D8-A27N-01A.RData
TCGA-E2-A14P-01A    local_rdata/TCGA-E2-A14P-01A.RData
TCGA-C8-A132-01A    local_rdata/TCGA-C8-A132-01A.RData
TCGA-D8-A140-01A    local_rdata/TCGA-D8-A140-01A.RData
TCGA-A8-A07B-01A    local_rdata/TCGA-A8-A07B-01A.RData
TCGA-A2-A0T3-01A    local_rdata/TCGA-A2-A0T3-01A.RData
TCGA-A8-A09I-01A    local_rdata/TCGA-A8-A09I-01A.RData
TCGA-EW-A1OZ-01A    local_rdata/TCGA-EW-A1OZ-01A.RData
TCGA-AN-A049-01A    local_rdata/TCGA-AN-A049-01A.RData
TCGA-AO-A12D-01A    local_rdata/TCGA-AO-A12D-01A.RData
TCGA-EW-A1IW-01A    local_rdata/TCGA-EW-A1IW-01A.RData
TCGA-EW-A1PH-01A    local_rdata/TCGA-EW-A1PH-0

In [25]:
# wm_pairs_df

In [43]:

# make a column with the path to the local_rdata files
# create a function that looks for the file names in the local_rdata path and check for the pair_id or other

precalled_purity_reviewer = PrecalledPurityReviewer()
precalled_purity_reviewer.set_review_data(
    data_path = 'precalled_purity_reviewer_output', 
    description= 'BRCA purity review', 
    df=wm_pairs_df,
    index=wm_pairs_df.index,
)

precalled_purity_reviewer.set_review_app(
    sample_info_cols=['absolute_highres_plot_WGS', 'hapaseg_allelic_segmentation_plot_WGS'],
    purity_col="PCA__ABSOLUTE__Cancer_DNA_fraction",
    acs_col='hapaseg_segfile_WGS', 
    rdata_fn_col='local_absolute_rdata',
    mut_fig_hover_data=['Hugo_Symbol', 'Chromosome', 'Start_position'],
)

precalled_purity_reviewer.set_default_review_data_annotations_configuration()
precalled_purity_reviewer.set_default_autofill()
# if you have a pickle error restart kernel and rerun all the cells

In [44]:
precalled_purity_reviewer.run(port=8099, mode='tab')

Setting auto_export_path to precalled_purity_reviewer_output/data.auto_export
Using precalled_purity_reviewer_output/data.auto_export for auto exporting.


/Users/odias/PurityReviewer/purity_venv/lib/python3.9/site-packages/AnnoMate/ReviewDataApp.py:738: UserWarning:

You are in test mode. Your data will not be saved.



Dash app running on http://0.0.0.0:8099/


<IPython.core.display.Javascript object>

New version!
'NULLType' object is not subscriptable
excepted error and initialized empty dataframes
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File ~/PurityReviewer/PurityReviewer/AppComponents/AbsolutePrecalledSolutionComponent.py:152, in gen_custom_precalled_absolute_component(
    data=<AnnoMate.DataTypes.GenericData.GenericData object>,
    data_id='TCGA-D8-A27T-01A',
    slider_value=5,
    selected_row_array=[0],
    manual_input_source='Use slider',
    rdata_fn_col='local_absolute_rdata',
    acs_col='hapaseg_segfile_WGS',
    mut_fig_hover_data=['Hugo_Symbol', 'Chromosome', 'Start_position'],
    purity_col='PCA__ABSOLUTE__Cancer_DNA_fraction',
    step_size=5,
    custom_parse_absolute_soln=<function my_custom_parse_absolute_soln>
)
    151 try:
--> 152     absolute_rdata_df,maf,maf_annot_list = parse_absolute_soln_func(data_sample[rdata_fn_col])
        data_sample = 

In [28]:
# matched_reviewer = MatchedPurityReviewer()
# matched_reviewer.set_review_data(
#     data_path = 'purity_reviwer_output', 
#     description= 'BRCA purity review', 
#     df=wm_pairs_df,
#     index=wm_pairs_df.index,
# )

# matched_reviewer.set_review_app(
#     sample_info_cols=['absolute_highres_plot_WGS', 'hapaseg_allelic_segmentation_plot_WGS'],
#     acs_col='hapaseg_segfile_WGS', 
#     maf_col='absolute_downsample_MAF_WGS',
#     rdata_fn_col='local_absolute_rdata',
#     mut_fig_hover_data=['Hugo_Symbol', 'Chromosome', 'Start_position'],
#     csize=CSIZE_DEFAULT,
#     # custom_parse_absolute_soln=my_custom_parse_absolute_soln
# )

# matched_reviewer.set_default_review_data_annotations_configuration()
# matched_reviewer.set_default_autofill()

## Run the reviewer

In [29]:
# matched_reviewer.run(port=8099, mode='tab')